In [1]:
import torch
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import Dataset
import json

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


c:\Users\Vedant\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.7.1+cu126 with CUDA 1208 (you have 2.7.1+cu118)
    Python  3.9.13 (you have 3.10.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
import torch
print(torch.version.cuda)  

11.8


In [3]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


True
NVIDIA GeForce RTX 3050 6GB Laptop GPU


In [4]:
# 1. Load Gemma 3 4B with Unsloth
max_seq_length = 2048
dtype = None  # Auto detection
load_in_4bit = True  # Use 4bit quantization to save memory

In [5]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/gemma-3-4b-it-bnb-4bit",  # Unsloth's optimized version
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

c:\Users\Vedant\AppData\Local\Programs\Python\Python310\lib\site-packages\unsloth_zoo\gradient_checkpointing.py:339: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"{DEVICE_TYPE}:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.7.5: Fast Gemma3 patching. Transformers: 4.53.2.
   \\   /|    NVIDIA GeForce RTX 3050 6GB Laptop GPU. Num GPUs = 1. Max memory: 6.0 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.7.1+cu118. CUDA: 8.6. CUDA Toolkit: 11.8. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [6]:
# 2. Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Rank
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                   "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,  # Unsloth optimizes dropout
    bias="none",
    use_gradient_checkpointing="unsloth",  # Memory efficient
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

Unsloth: Making `base_model.model.model.vision_tower.vision_model` require gradients


In [7]:
# 3. Prepare your dataset
def format_data(examples):
    """Format Daily Mail/CNN data for instruction following"""
    texts = []
    for article, summary in zip(examples['article'], examples['summary']):
        text = f"""<bos><start_of_turn>user
Please summarize the following news article:

{article}<end_of_turn>
<start_of_turn>model
{summary}<end_of_turn><eos>"""
        texts.append(text)
    return {"text": texts}

In [21]:
# Load Daily Mail/CNN dataset
# Assuming you have a JSON file with 'article' and 'summary' fields
def load_news_data(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except UnicodeDecodeError:
        # Try with different encoding if UTF-8 fails
        with open(file_path, 'r', encoding='latin1') as f:
            data = json.load(f)
    
    # Convert to HuggingFace dataset format
    dataset = Dataset.from_dict({
        'article': [item['article'] for item in data],
        'summary': [item['summary'] for item in data]
    })
    
    return dataset.map(format_data, batched=True)

In [22]:
train_dataset = load_news_data("C:\LLM\daily_mail_cnn_train_clean.json")

Map: 100%|██████████| 10000/10000 [00:00<00:00, 38904.99 examples/s]


In [23]:
# 4. Training setup
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,  # Can make training 5x faster for short sequences
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=100,  # Adjust based on your dataset size
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_steps=25,
        save_total_limit=2,
    ),
)


UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 24432: character maps to <undefined>